# Option PNL Exercise

The goal of this exercise is to calculate the delta-hedged PNL of a short option position.  Rather than using simulated data, we will use real-world data.

The file `option_pnl.csv` contains the price data for a particular option that traded in the market.  Your assignment is the calculate the PNL of selling this option starting on 1/2/2018 and then delta-hedging it through its expiration on 12/31/2018.  You can assume that the risk-free rate and dividend yield are both zero.

Answer the following questions:
1. What was the implied volatility of the option at the time of execution?
1. What was the realized volatility of the underlying over the life of the delta-hedged option position?
1. Based on the difference between implied volatility and realized volatility, would you expect to make or lose money?
1. What is the cumulative PNL for holding this delta-hedged option position?
1. How does the actual PNL compare with the PNL that would be predicted by the vega at the time of execution?  What could explain the discrepancies?

**Note:** Your deliverable will be a Jupyter Notebook containing all your code.

In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# Load data
df = pd.read_csv('/Users/yuanhanlim/Desktop/01_option_pnl/option_pnl.csv')

# Step 1: Implied volatility at the time of execution
implied_vol = df['volatility'].iloc[0]
print(f"Implied volatility at execution: {implied_vol:.4f}")

# Step 2: Realized volatility calculation using log returns
df['log_return'] = np.log(df['upx'] / df['upx'].shift(1))
realized_vol = df['log_return'].std() * np.sqrt(252)  # annualized volatility
print(f"Realized volatility: {realized_vol:.4f}")

# Step 3: PNL expectation based on volatility difference
if realized_vol < implied_vol:
    print("Expect to make money since realized volatility is lower than implied.")
else:
    print("Expect to lose money since realized volatility is higher than implied.")

# Function to calculate d1
def calculate_d1(S, K, T, sigma, r=0):
    return (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / ((sigma * np.sqrt(T))+0.000000001)

# Function to calculate put option delta
def calculate_put_delta(S, K, T, sigma, r=0):
    d1 = calculate_d1(S, K, T, sigma, r)
    return norm.cdf(d1) - 1  # Delta for a put option

# Apply Black-Scholes formula to each row in the DataFrame
df['delta'] = df.apply(lambda row: calculate_put_delta(row['upx'], row['strike'], row['t2x'], row['volatility']), axis=1)

# Step 4: Option PNL (simply the daily change in option price)
df['option_pnl'] = df['buy_sell']* df['option_price'].diff()

# Step 5: Calculate daily PNL from delta hedging (hedge PNL)
df['delta_hedged_pnl'] = df['delta'].shift(1) * df['upx'].diff()

# Step 6: Total PNL calculation (Option PNL + Delta-Hedged PNL)
df['total_pnl'] = df['option_pnl'] + df['delta_hedged_pnl']
print(f"Total PNL (option PNL + delta hedging PNL): {df['total_pnl'].sum():.6f}")

# Step 7: Vega-based PNL prediction
# Vega is typically calculated as: Vega = S * sqrt(T) * N'(d1), we calculate it at execution
def calculate_vega(S, K, T, sigma, r=0):
    d1 = calculate_d1(S, K, T, sigma, r)
    return S * np.sqrt(T) * norm.pdf(d1)

# Vega at execution (first row)
vega_at_execution = calculate_vega(df['upx'].iloc[0], df['strike'].iloc[0], df['t2x'].iloc[0], df['volatility'].iloc[0])

# Volatility difference
vol_diff = implied_vol - realized_vol

# Predicted PNL from Vega
predicted_pnl_from_vega = vega_at_execution * vol_diff
print(f"Predicted PNL based on Vega: {predicted_pnl_from_vega:.6f}")

Implied volatility at execution: 0.1327
Realized volatility: 0.1723
Expect to lose money since realized volatility is higher than implied.
Total PNL (option PNL + delta hedging PNL): -3.752832
Predicted PNL based on Vega: -4.236322


1. Delta Hedging Imperfection: Delta hedging assumes continuous rebalancing, but in practice, you can only hedge discretely at each trading interval. Small mismatches in timing or price movements between hedging adjustments can lead to deviations from the expected Vega-based PNL.
2. Model Assumptions: The Black-Scholes model assumes constant volatility, no transaction costs, and continuous rebalancing. However, in reality, volatility is not constant, which can cause differences between realized and implied volatility over time. Moreover, transaction costs, bid-ask spreads, and liquidity constraints might also affect actual PNL, making it different from what the theoretical model predicts.

In [46]:
import numpy as np
from scipy.stats import norm

def black_scholes_vega(S, K, T, r, sigma):
    # d1 formula
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    
    # Calculate Vega (S * sqrt(T) * N'(d1))
    vega = S * np.sqrt(T) * norm.pdf(d1)
    
    return vega

# Inputs for the Vega calculation
S = 269.677         # Spot price of SPY on 1/2/18
K = 269             # Strike price
T = 0.996031746     # Time to expiration (t2x on 1/2/18)
r = 0               # Risk-free rate
sigma = 0.1327      # Implied volatility on 1/2/18

# Calculate Vega
vega = black_scholes_vega(S, K, T, r, sigma)
print(f"Vega at time of execution: {vega}")


Vega at time of execution: 106.98290121798476
